In [1]:
import certifi
import time
import datetime
import requests
import pandas as pd
import urllib3
import MySQLdb
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import matplotlib.pyplot as plt

In [2]:
# set up db connection
db = MySQLdb.connect(host="localhost",    
                     user="root",        
                     passwd="root",  
                     db="pipa_db")
cur = db.cursor()

## 1. Extract and parse data from PubMed using a crawler

In [3]:
# set up extraction variables
dailyMax = 200
batchNum = 20
sleeptimeforPubmed = 5
backstep = 20

cur.execute('select pmid from articles order by pmid desc')
rows = cur.fetchall()
startingPMID = rows[backstep][0]
endingPMID = rows[0][0]

# for testing
# startingPMID = 28315832

print "Starting PMID is %d" % startingPMID

Starting PMID is 28316328


In [4]:
# Main extraction loop
cur.execute('truncate table articles')
print "Start extracting ..."
for i in range (startingPMID, startingPMID + dailyMax, batchNum):

    
    # Build extraction url
    PMIDs = ""
    for j in range (batchNum):
        PMIDs = PMIDs + str(i + j ) +","
    url = "https://www.ncbi.nlm.nih.gov/pubmed/"+ PMIDs +"?report=medline&format=text"
    
    # Extract data and clean up non-unicode
    time.sleep(sleeptimeforPubmed) # sleep 5 sec between extraction batches
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
    r = http.request('GET', url)
    soup = BeautifulSoup(r.data, "lxml")
    allInfo = unicode(soup.pre.string)

    # Parse data to fetch PMID, Title, Abstract (data are messy -- wrapped with try-except)
    if allInfo.find('PMID- ') < 0:
        print url
        print 'No new data found.'
        break
    
    contentOffset = 6
    while (allInfo.find('AB  -') > 0 and allInfo.find('TI  -') > 0):
        IdIndexStart = allInfo.index('PMID- ') + contentOffset
        IdIndexEnd = allInfo.index('\n', IdIndexStart)
        PMID = allInfo[IdIndexStart:IdIndexEnd]

        try:
            tiIndexStart = allInfo.index('TI  -') + contentOffset
            if allInfo.find('PG  -', tiIndexStart) > 0:
                tiIndexEnd = allInfo.index('PG  -', tiIndexStart) - 1
            elif allInfo.find('LID -', tiIndexStart) > 0:
                tiIndexEnd = allInfo.index('LID -', tiIndexStart) - 1
            else:
                tiIndexEnd = allInfo.index('AB  -', tiIndexStart) - 1
            ti = allInfo[tiIndexStart:tiIndexEnd]
        except:
            ti =""

        abIndexStart = allInfo.index('AB  -') + contentOffset
        try:
            abIndexEnd = allInfo.index('FAU -', abIndexStart) - 1
        except:
            abIndexEnd = abIndexStart + 1000
        ab = ti + "        Abstract:" + allInfo[abIndexStart:abIndexEnd]
        
        # load data into the Articles table in MySQL db
        insert_articles_sql = "insert into articles (pmid, abstract) values ("+ PMID +", '"+ ab + "')"
        try:
            cur.execute(insert_articles_sql)
            db.commit()
        except:
            db.rollback()
        
        endingPMID = int(PMID)
        if (endingPMID % 10 == 0):
            print "%d extracted" % endingPMID
        allInfo = allInfo[abIndexEnd:]
    
now = datetime.datetime.now()
insert_last_pmid_sql = 'insert into last_pmid (record_date, pmid) values ("%s", %d)' % (now, endingPMID)
try:
    cur.execute(insert_last_pmid_sql)
    db.commit()
except:
    db.rollback()
    
print "Extraction and loading completed: PMID %d ~ %d" % (startingPMID, endingPMID) 

Start extracting ...
28316330 extracted
28316340 extracted
28316350 extracted
28316360 extracted
28316370 extracted
28316380 extracted
28316390 extracted
28316400 extracted
28316410 extracted
28316420 extracted
28316430 extracted
28316440 extracted
28316460 extracted
28316470 extracted
28316480 extracted
28316490 extracted
28316500 extracted
28316510 extracted
28316520 extracted
Extraction and loading completed: PMID 28316328 ~ 28316527


## 2. Train the recommendation engine

In [6]:
def import_data_articles(table_name, cur):
    """
    Import article data from a table in MySQL db and load them into a pandas Dataframe.
    
    :type table_name: str
    :type cur: db cursor
    :rtype: pandas DataFrame
    """
    #ds = pd.read_csv(data_source)
    ds = cur.execute('select pmid, abstract from ' + table_name)
    r = cur.fetchall()
    return pd.DataFrame(list(r),columns=['PMID', 'abstract'])

In [7]:
def import_user_articles(userId, cur):
    """
    Import a user's articles given id and load them into a pandas Dataframe.
    
    :type table_name: str
    :type cur: db cursor
    :rtype: pandas DataFrame
    """
    
    cur.execute('select pmid, title, abstract from user_articles where user_id=%d'% (userId))
    r = cur.fetchall()
    userdata= pd.DataFrame(list(r),columns=['PMID', 'title', 'abstract'])
    userdata['abstract'] = userdata['title']+'        Abstract:'+userdata['abstract']
    userpool = userdata[['PMID', 'abstract']]
    return userpool

In [8]:
def import_recommend_articles(table_name, cur):
    """
    Import recommend articles from a table in MySQL db and load them into a pandas Dataframe.
    
    :type table_name: str
    :type cur: db cursor
    :rtype: pandas DataFrame
    """
    
    ds = cur.execute('select source_pmid, title_1, title_2, title_3, title_4, title_5, \
    score_1, score_2, score_3, score_4, score_5 from %s where date(entry_date) = curdate()' % table_name)
    r = cur.fetchall()
    colnames = ['Source_PMID', 'title_1', 'title_2', 'title_3', 'title_4', 'title_5', \
                'score_1', 'score_2', 'score_3', 'score_4', 'score_5']
    
    return pd.DataFrame(list(r),columns=colnames)

In [9]:
def train(userpool, dailypool, userId, cur, db):
    """
    Find the top 10 similar articles for each entry in the user's list and write into database
    :type userpool: pandas DataFrame
    :type dailypool: pandas DataFrame
    :type userId: int
    :type cur: db cursor
    :type db: MySQL db
    :rtype: void
    """
    
    ds = pd.concat([userpool, dailypool])
    ds.reset_index(drop=True, inplace=True)

    # clean up not ASCII chars
    ds['abstract'] = ds["abstract"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))

    # calculate term frequency
    tf = TfidfVectorizer(analyzer='word',
                         ngram_range=(1, 3),
                         min_df=0,
                         stop_words='english')
    tfidf_matrix = tf.fit_transform(ds['abstract'])

    # calculate similarity of papers(from users and pools) based on term frequency matrix
    cosine_similarities = linear_kernel(tfidf_matrix[:userpool.shape[0]], tfidf_matrix[userpool.shape[0]:])
    similarities = pd.DataFrame(cosine_similarities)

    prediction_matrix = pd.DataFrame(data=userpool.PMID, columns=['PMID'], index=userpool.index)
    similarity_scores = prediction_matrix.copy()
    abstract_matrix = prediction_matrix.copy()

    top_n = 12
    if (ds.shape[0] < top_n):
        top_n = ds.shape[0] + 1

    for i in range(1,top_n-1):
        col_name = i
        prediction_matrix[col_name] = None
        similarity_scores[col_name] = -1
        abstract_matrix[col_name] = None

    now = datetime.datetime.now()

    for idx in range(userpool.shape[0]):
        
        # sort papers by their similarity score
        # print "idx: %s" % idx
        similar_indices = cosine_similarities[idx].argsort()[:-top_n:-1]
        
        # retrive the ID and abstract of sorted papers
        # print "similar indices: %s" % similar_indices
        similar_items = [[cosine_similarities[idx][i], ds['PMID'][i], ds['abstract'][i]]for i in similar_indices]
        
        # save top 10 hits into MySQL database
        for i in range(0,top_n-2):
            prediction_matrix.ix[idx, i] = similar_items[i][1] # insert into db directly
            similarity_scores.ix[idx, i] = similar_items[i][0] # insert into a db table with date

            titleEnd = similar_items[i][2].index('        Abstract:')           
            similar_items[i][2] = similar_items[i][2][:titleEnd][:200].replace('\"', ' ')

        query = 'insert into recommend_articles (\
        entry_date, user_id, source_pmid,\
        pmid_1, pmid_2, pmid_3, pmid_4, pmid_5, pmid_6, pmid_7, pmid_8, pmid_9,pmid_10, \
        score_1, score_2, score_3,score_4, score_5, score_6, score_7, score_8, score_9, score_10,\
        title_1, title_2, title_3, title_4, title_5, title_6, title_7, title_8, title_9, title_10 ) \
        values \
        ("%s", %d, %d,\
            %d, %d, %d, %d, %d,\
            %d, %d, %d, %d, %d,\
            %.6f,%.6f,%.6f,%.6f,%.6f,\
            %.6f,%.6f,%.6f,%.6f,%.6f,\
            "%s","%s","%s","%s","%s",\
            "%s","%s","%s","%s","%s")'\
        %(now, userId,userpool.ix[idx,'PMID'],\
         similar_items[0][1],similar_items[1][1],similar_items[2][1],\
         similar_items[3][1],similar_items[4][1],similar_items[5][1],\
         similar_items[6][1],similar_items[7][1],similar_items[8][1],\
         similar_items[9][1],\
         similar_items[0][0],similar_items[1][0],similar_items[2][0],\
         similar_items[3][0],similar_items[4][0],similar_items[5][0],\
         similar_items[6][0],similar_items[7][0],similar_items[8][0],\
         similar_items[9][0],\
         similar_items[0][2],similar_items[1][2],similar_items[2][2],\
         similar_items[3][2],similar_items[4][2],similar_items[5][2],\
         similar_items[6][2],similar_items[7][2],similar_items[8][2],\
         similar_items[9][2])
        # print query
        try:
            cur.execute(query)
            db.commit()
        except:
            db.rollback()

In [10]:
dailypool = import_data_articles('articles', cur)
dailypool.head()

,PMID,abstract
0,28316328,Reoperation After Cyst Excision with Hepaticoj...
1,28316331,Targeted therapies: Ibrutinib: new option for ...
2,28316336,Flint Water Crisis: What Happened and Why? ...
3,28316338,Detection and quantification of 2H and 3R phas...
4,28316344,Mechanoluminescent Imaging of Osmotic Stress-I...


In [11]:
cur.execute('select distinct user_id from user_articles')
userIdlist = cur.fetchall()


((1L,),)


In [12]:
start = time.time()
for id in userIdlist:
    userpool = import_user_articles(id[0], cur)       
    train(userpool, dailypool, id[0], cur, db)
    
print("Engine trained in %s seconds." % (time.time() - start))

Engine trained in 0.354042053223 seconds.


In [13]:
userpool.head()

,PMID,abstract
0,1,p53 regulates mitochondrial respiration ...
1,2,Life with oxygen Abstract:The survival ...
2,3,Autophagy and metabolism Abstract:Autop...


In [14]:
recommendations = import_recommend_articles('recommend_articles', cur)
recommendations

,Source_PMID,title_1,title_2,title_3,title_4,title_5,score_1,score_2,score_3,score_4,score_5
0,1,[Effects of Meek skin grafting on patients wit...,Predictive factors of neoplastic gallbladder p...,A rare cause of upper gastrointestinal hemorrh...,[Analysis of corneal complications in children...,"[The compilation, characteristic and value of ...",0.034558,0.033082,0.031156,0.028903,0.020382
1,2,PEEK and CFR-PEEK as alternative bearing mater...,Uranium Tetrakis-Aryloxide Derivatives Support...,[Clinical observation on the treatment of untr...,Effects of different strength training frequen...,WebMolCS: a Web-Based Interface for Visualizin...,0.027130,0.016563,0.016299,0.016117,0.015802
2,3,[Effects of Meek skin grafting on patients wit...,[Analysis of corneal complications in children...,Predictive factors of neoplastic gallbladder p...,[Association between food intake and the serum...,Plant protein-based feeds and commercial feed ...,0.032004,0.029425,0.019105,0.017132,0.016699
3,1,[Effects of Meek skin grafting on patients wit...,Predictive factors of neoplastic gallbladder p...,A rare cause of upper gastrointestinal hemorrh...,[Analysis of corneal complications in children...,"[The compilation, characteristic and value of ...",0.034558,0.033082,0.031156,0.028903,0.020382
4,2,PEEK and CFR-PEEK as alternative bearing mater...,Uranium Tetrakis-Aryloxide Derivatives Support...,[Clinical observation on the treatment of untr...,Effects of different strength training frequen...,WebMolCS: a Web-Based Interface for Visualizin...,0.027130,0.016563,0.016299,0.016117,0.015802
5,3,[Effects of Meek skin grafting on patients wit...,[Analysis of corneal complications in children...,Predictive factors of neoplastic gallbladder p...,[Association between food intake and the serum...,Plant protein-based feeds and commercial feed ...,0.032004,0.029425,0.019105,0.017132,0.016699
6,1,Seroepidemiology of Toxoplasma gondii infectio...,ZNF395 Is an Activator of a Subset of IFN-Stim...,Understanding the Electrochemical Formation an...,Reporting adverse transfusion reactions: A ret...,Prevalence of ectoparasites in dogs of Shimoga...,0.051001,0.043591,0.023963,0.022421,0.019028
7,2,Effect of Fermented Spinach as Sources of Pre-...,PEEK and CFR-PEEK as alternative bearing mater...,The short-term impacts of development-induced ...,Comparison of deferasirox and deferoxamine eff...,Dyslipidemia rather than Type 2 Diabetes Melli...,0.035816,0.028319,0.023075,0.019488,0.018424
8,3,ZNF395 Is an Activator of a Subset of IFN-Stim...,Seroepidemiology of Toxoplasma gondii infectio...,Reporting adverse transfusion reactions: A ret...,Inducible spy Transcription Acts as a Sensor f...,Understanding the Electrochemical Formation an...,0.044311,0.039320,0.029772,0.018328,0.017919


### Summary

* Crawl publication resources from Pubmed with urllib3
* Parsing information on the webpage with beautifulsoup4
* Calculate similarity with sklearn TF-IDF algorithm
* Read and write data accessing MySQL database

### What's next

* Scale up to more data sources
* Improve recommendation accuracy
* Include collaborative filtering
* Add more features to front-end
